# BackTestのチュートリアル

## 1. 実装するBotのロジック

Ukiさんの「オープニングレンジ・ブレイクアウト」をお借りして、バックテストのtutorialを作成しました。

Binance用に書いていますが、PublicAPIのみ利用のためアカウントは不要です。

<blockquote class="twitter-tweet"><p lang="ja" dir="ltr">お待ちかねのロジックを公開。<br>(元祖)ドテン君はブレイクアウトだが通常のHLチャネルブレイクではない。その手法は「オープニングレンジ・ブレイクアウト」と呼ばれるものである。文章での説明は面倒なのでhohetoとの会議資料をそのまま添付する。ストラテジーに関する質問には応対できません。 <a href="https://t.co/LB6mdxVZZo">pic.twitter.com/LB6mdxVZZo</a></p>&mdash; UKI (@blog_uki) <a href="https://twitter.com/blog_uki/status/981768546429448192?ref_src=twsrc%5Etfw">April 5, 2018</a></blockquote> <script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>

![](https://pbs.twimg.com/media/DZ_xJQFUMAEOrpS?format=jpg&name=900x900)

## 2. BreakOutAgentの実装

日本語で読み解くと以下のようになります。
「シグナル点灯した次の足の始値」ではなく、シグナル点灯直後の最良値でエントリーすることに変更した以外はほぼ忠実に書き下している（つもり）

1) `Agent.on_clock`を10分毎に呼び出すように設定する(`Agent.clock_interval`)
2) `Agent.on_clock`内で以下の処理を行う。
   1) 前処理
      1) 前回のon_clock中でのオーダーが処理中の場合はなにもしない（リターン）
   2) Long/Short判定
      1) 現在時刻から2時間足を6本取得する。`session.ohlcv`（6本目の最後の足は未確定足。10分毎に呼ばれるたびにupdateされる）
      (ohlcv[0]-ohlcv[5]へ値を格納)
      2) 1-5本目の足のレンジ幅（高値ー安値）の平均値を計算する。`(ohlcv['high]-ohlcv['low']).mean()`
      3) 最新足（未確定）の始値〜高値を計算する
      4) 最新足（未確定）の始値〜安値を計算する
      5) 2,3,4の結果からLong/Short判定
    3) オーダー執行
       1) Long判定のとき
          1) すでにLongポジションがある場合は何もしない。
          2) Shortポジションがあった場合はドテン（最良値、倍サイズ）
          3) 上記に当てはまらない場合は最良値、通常サイズでLong
        3) Short判定のとき
           1) すでにShortポジションがある場合は何もしない
           2) Shortポジションがあった場合はドテン（最良値、倍サイズ）
           3) 上記に当てはまらない場合は最良値、通常サイズでShort

## 依存ライブラリのインストール

### 依存ライブラリ
* numpy
* pandas
* bokeh

それぞれpipでインストールしてください。
あるいはまとめて
```
python3 -m pip install -r requirement.txt
```
でもOK.


#
#
#

In [1]:
# 必要ライブラリーのimport
import pandas as pd

# rbotは今回提供するbacktestライブラリ
import rbot
from rbot import BaseAgent
from rbot import BinanceMarket
from rbot import BackRunner
from rbot import time_string
from rbot import Market
from rbot import DAYS_BEFORE
from rbot.chart import Chart

## break out Agent(bot)の実装

### 時間の扱い



### 必須メソッド



### `clocl_interval`


### `on_clock`


### `on_update`


### `on_tick`



In [2]:
class BreakOutAgent(BaseAgent):
    """
        Agentのクラス名は任意。
        BaseAgentを継承し、clock_interval, on_clockを実装する。
    """

    def __init__(self, param_K=1.6):
            """ super().__init()__ で上位クラスの初期化を必ずすること　"""
            super().__init__()
            self.param_K = param_K  # パラメターKを設定する。

    def clock_interval(self):
        """ on_clockが呼び出される間隔を秒で返す。今回は10分毎にする"""
        return 60 * 10
    
    def on_clock(self, time_us, session):
        """ Botのメインロジック。on_clockで設定した秒数毎に呼ばれる """
        # 前処理/ 前回のon_clock中でのオーダーが処理中の場合はなにもしない（リターン）        
        if session.short_order_size or session.long_order_size:
            return 
        
        ############   メインロジック  ###################### 
        ohlcv_df = session.ohlcv(60*60*2, 6)  # 2時間足(60*60*2sec)を６本取得。 最新は６番目。ただし未確定足
        if len(ohlcv_df.index) < 6:           # データが過去６本分そろっていない場合はなにもせずリターン
            return 

        ohlcv5 = ohlcv_df[:-2]       # 過去５本足（確定）
        range_width = (ohlcv5['high'] - ohlcv5['low']).mean() * self.param_K  # 価格変動レンジの平均を計算 * K

        # Long/Short判定
        ohlcv_latest = ohlcv_df[-2:-1]     # 最新足１本(未確定)        
        diff_low   =   (ohlcv_latest['open'][0] - ohlcv_latest['low'][0])
        detect_short  = range_width < diff_low

        diff_high  = - (ohlcv_latest['open'][0] - ohlcv_latest['high'][0])  
        detect_long = range_width  < diff_high
        
        ##########  メインロジック中に利用したindicatorのロギング（あとでグラフ化するため保存）    ##############
        self.log_indicator('diff_low', time_us, diff_low)
        self.log_indicator('diff_high', time_us, diff_high)
        self.log_indicator('range_width', time_us, range_width)

        ##########　執行戦略（順方向のポジションがあったら保留。逆方向のポジションがのこっていたらドテン）#########
        ORDER_SIZE = 0.01     # 標準オーダーサイズ(ドテンの場合はx2)
        ORDER_LIFE = 60*10    # オーダーの有効期間（60x10秒=10分)
        
        if detect_long and (not session.long_position_size): 
            if session.short_position_size: # Shortポジションがあった場合はドテン
                session.place_order( #オーダーを発行する
                            'Buy',                      # 'Buy', 'Sell'を選択
                            session.best_buy_price,     # 最後にbuyがtakeされた価格。sell側にはbest_sell_priceを提供。
                                                        # 任意の価格が設定できるがtakeになる価格でもmakeの処理・手数料で処理している。
                            ORDER_SIZE * 2,             # オーダーサイズ BTCBUSDの場合BTC建で指定。ドテンなので倍サイズでオーダー
                            ORDER_LIFE,                 # オーダーの有効期限（秒）。この秒数をこえるとExpireする。
                            'doten Long'                # あとでログで識別できるように任意の文字列が設定できる。
                )
            else:
                session.place_order('Buy', session.best_buy_price, ORDER_SIZE, ORDER_LIFE, 'Open Long')    

        if detect_short and (not session.short_position_size): # short判定のとき
            if session.long_position_size:  # Longポジションがあった場合はドテン
                session.place_order('Sell', session.best_sell_price, ORDER_SIZE * 2, ORDER_LIFE, 'Doten Short')
            else:
                session.place_order('Sell', session.best_sell_price, ORDER_SIZE, ORDER_LIFE, 'Open Short') 


## 3. バックテスト用データのロード
現在はBianceのみ対応。

`Market.open`でバイナンス（`BN`)の通貨ペアー(`BTCBUSD`)を開く。
その後、`Market.download`で過去n日分のデータをダウンロードする。

2回目以降はディフォルトではダウンロードしない。
再ダウンロードしたい場合は、`Market.download(10, True)`のように再ダウンロードフラグをTrueにする。

In [3]:
binance = Market.open('BN', 'BTCBUSD')  # binance marketはあとで利用するので保存しておく

Market.download(10)             # １０日前より最新のログデータをダウンロード（差分）
#Market.download(10, True)       # 再ダウンロード (１日あたり１０秒＋アルファかかります。)


Start download from 2022-12-06T05:31:51.717521


In [4]:
# Marketオブジェクトを表示するとデータの保持期間、保存場所がわかる
# かなり巨大なデータベースになるので、不要になったら場所を確認して手で削除してください。
# 手元の計測では、70日分で15Gありました（１日あたり200Mb)
binance

start,end
1653782401756000,1671148799995000
"""2022-05-29T00:00:01.756000""","""2022-12-15T23:59:59.995000"""
days=,200


In [5]:
# DBのある場所を表示します。かなり大きなファイルになりますので不要になった場合手動で消してください。
binance.file_name

'/Users/takeoyasukawa/Library/Application Support/net.takibi.rbot/DB/BN/BTCBUSD.db'

## 4. バックテスト実行

In [6]:
 
back_runner = BackRunner(
        'BN',           # Binance は BNと省略します。
        'BTCBUSD',      # 通貨ペアーを選択します。
        False           # 注文時に指定するサイズが通貨ペアーの右側通貨の場合True。BinanceのBTCBUSDはBTCでサイズを指定するのでFalse
)

back_runner.maker_fee_rate = 0.1 * 0.01  # maker_feeを指定（0.1%）. takerは未実装。現在は相手板にぶつける注文をしてもmaker_feeが適用される。

In [10]:
agent = BreakOutAgent()         # Agentのインスタンスを作ります（あとで利用するので変数に保存しておきます）。

# 10日前から最新までバックテストする例（最新データは２日ぐらい前のため８日間のバックテスト）。手元のM1 MacMiniで１日あたり数秒かかります。
# まれにまだエラーがでることがあります（再現条件が突き止められていない）。その場合はPythonカーネルから再起動してみてください。
back_runner.run(
    agent,                      # backtest するagentインスタンスを指定します。
    rbot.DAYS_BEFORE(10),       # 開始時刻を指定します(us)。0だとDBにある最初のデータから処理。DAYS_BEFOREはN日まえのtimestampを返すユーティリティ関数です。
    0                           # 終了時刻を指定します(us). 0だとDBにある最後のデータまで処理。
)     

back_runner                     # back testの結果概要が表示されます

Back testing... 2022-12-15T18:00 /   8[days] / rec=  21129193 / on_tick=     0 / on_clock=  3754 / on_update=   0 /   42644 xSpeed /  1385550[rec/sec]

実行結果の分析

実行結果は BackRunner.resultに保存されています。pd.DataFrame形式になっていますのでpandasの機能をつかって分析します。

なお、`size`とはオーダー発行したときの単位での大きさ、`volume`は反対側の通貨での大きさという意味で使い分けています。
またドテンの場合、一つのオーダーがポジションのクローズと新規のポジション作成の２つに分けられ、`sub_id`が一つインクリメントされます。

各カラムの意味は以下のとおり

* index(`create_time`)        オーダーが作られた時刻
* `update_time`	            更新時刻（ステータスが変化したときに更新）
* `1order_id`	                オーダーID（自動付与）
* `sub_id`	                オーダーSubID（ドテンの場合、オーダーCloseとOpenに分割されそれぞれIDが付与される）
* `order_side`	            'Buy'または'Sell'
* `status`	                'Open', 'Close', 'Expire'(期限切れでオーダー無効)のどれかの状態をとる
* `open_price`	            ポジションをオープンしたときの価格
* `open_size`	                                        サイズ
* `open_volume`	                                    ボリューム
* `close_price`	            ポジションをクローズしたときの価格
* `close_size`                                        サイズ
* `close_volume`                                      ボリューム
* `order_price`	            オーダで指定した価格
* `order_size`	                                    サイズ
* `order_volume`	                                    ボリューム
* `profit`	                取引による損益
* `fee`	                    取引手数料
* `total_profit`            手数料込みの損益
* `position_change`	        ポジションの変化
* `message`	                オーダー時に指定したメッセージ文字列
* `sum_profit`	            累積損益
* `position`                  現在のポジション






In [11]:
pd.options.display.max_rows=30

df = back_runner.result
df

,update_time,order_id,sub_id,order_side,status,open_price,open_size,open_volume,close_price,close_size,...,order_price,order_size,order_volume,profit,fee,total_profit,position_change,message,sum_profit,position
create_time,,,,,,,,,,,,,,,,,,,,,
2022-12-07 07:20:00+00:00,2022-12-07 07:30:00.030000+00:00,0000-0001,0,Sell,Expire,0.00,0.00,0.0000,0.00,0.00,...,16934.06,0.01,169.3406,0.0000,0.000000,0.000000,0.00,Open Short,0.000000,0.00
2022-12-07 07:50:00+00:00,2022-12-07 07:50:00.306000+00:00,0000-0002,0,Sell,Open,16763.22,0.01,167.6322,0.00,0.00,...,16763.22,0.01,167.6322,0.0000,0.167632,-0.167632,-0.01,Open Short,-0.167632,-0.01
2022-12-08 15:50:00+00:00,2022-12-08 15:49:59.963000+00:00,0000-0003,0,Buy,Close,16763.22,0.01,167.6322,16940.00,0.01,...,16940.00,0.01,169.4000,-1.7678,0.169400,-1.937200,0.01,doten Long,-2.104832,0.00
2022-12-08 15:50:00+00:00,2022-12-08 15:49:59.963000+00:00,0000-0003,1,Buy,Open,16940.00,0.01,169.4000,0.00,0.00,...,16940.00,0.01,169.4000,0.0000,0.169400,-0.169400,0.01,doten Long,-2.274232,0.01
2022-12-09 13:40:00+00:00,2022-12-09 13:50:00.032000+00:00,0000-0004,0,Sell,Expire,0.00,0.00,0.0000,0.00,0.00,...,17233.61,0.02,344.6722,0.0000,0.000000,0.000000,0.00,Doten Short,-2.274232,0.01
2022-12-11 20:30:00+00:00,2022-12-11 20:40:00.151000+00:00,0000-0005,0,Sell,Expire,0.00,0.00,0.0000,0.00,0.00,...,17136.00,0.02,342.7200,0.0000,0.000000,0.000000,0.00,Doten Short,-2.274232,0.01
2022-12-11 21:00:00+00:00,2022-12-11 21:10:00.079000+00:00,0000-0006,0,Sell,Expire,0.00,0.00,0.0000,0.00,0.00,...,17136.71,0.02,342.7342,0.0000,0.000000,0.000000,0.00,Doten Short,-2.274232,0.01
2022-12-11 21:30:00+00:00,2022-12-11 21:30:00.978000+00:00,0000-0007,0,Sell,Close,16940.00,0.01,169.4000,17121.56,0.01,...,17121.56,0.01,171.2156,1.8156,0.171216,1.644384,-0.01,Doten Short,-0.629848,0.00
2022-12-11 21:30:00+00:00,2022-12-11 21:30:00.978000+00:00,0000-0007,1,Sell,Open,17121.56,0.01,171.2156,0.00,0.00,...,17121.56,0.01,171.2156,0.0000,0.171216,-0.171216,-0.01,Doten Short,-0.801063,-0.01


### いろいろ分析

損益などの単位は、オーダー時に指定したサイズの単位の逆側になります。

例：今回はBTCBUSDの組み合わせに対し、BTCのサイズでオーダ発行しているので、BUSDでの損益。


In [12]:
# 損益結果　 
df['sum_profit'][-1]

1.0990252000000178

In [13]:
# 利益のピーク
df["sum_profit"].max()

1.2741218000000178

In [14]:
# 1回の取引の最大利益
df["total_profit"].max()

3.062403399999983

In [15]:
# １回の取引の最大損失
df["total_profit"].min()

-1.937199999999994

In [16]:
# 手数料なしの場合の損益
df['profit'].sum()

2.6418000000000177

In [17]:
# 手数料合計
df['fee'].sum()

1.5427748000000001

In [18]:
# オーダー回数・Expire数の分析
df.groupby(["order_side", "status"])["status"].count()

order_side  status
Buy         Close     2
            Expire    1
            Open      2
Sell        Close     2
            Expire    7
            Open      3
Name: status, dtype: int64

In [19]:
# 勝率分析
win_trade_df  = df[ (df["status"] == "Close") &  (0 < df["total_profit"])]
loss_trade_df = df[ (df["status"] == "Close") &  (df["total_profit"] < 0)]

win_trade = len(win_trade_df)
loss_trade = len(loss_trade_df)

print("買った数:", win_trade)
print("負けた数:", loss_trade)
print("勝率:", win_trade/(loss_trade+win_trade))

買った数: 2
負けた数: 2
勝率: 0.5


In [20]:
# 約定時間分析

df['exec_time'] = df['update_time'] - df.index
df[(df['status'] != 'Expire')]['exec_time'].mean()

Timedelta('0 days 00:01:12.418000')

## 6. グラフで確認

### まずは該当期間のOHLCV（ローソク足）チャートを書く

#### OHLCVデータの取得

まず「３.バックテスト用データのダウンロード」でデータをダウンロードしておきます。

`Market.ohlcv(開始timestamp[us], 終了timestamp[us], 足の幅[sec])`
で取得できる。以下の例では10日前のタイムスタンプを返す`rbot.DAYS_BEFORE`を利用している。

毎回Tickデータから任意幅のOHLCVを作っているのでかなり重い処理になってしまっていますが、任意の幅のOHLCVがつくれるのでこれだけ利用しても便利だと思う。

出力はpandas.DataFrameで帰ってきます。

なお、現在、高速化のためにキャッシュしているロジックの品質に問題があり「3.バックテスト用データのダウンロード」で指定した日付やohlcvでしてした期間を変更して再実行すると稀にエラーがでることがあります。その場合は①Pythonカーネルの再起動、②データの再ダウンロード、③対象期間の変更（一致）を試してみてください。

In [21]:
# すでに notebookの最初でbinanceインスタンスは以下のコードで作成済み。ここからohlcvを取り出す。
#binance = Market.open("BN", "BTCBUSD")
#Market.download(10)

rbot.init_debug_log()

ohlcv = binance.ohlcv(DAYS_BEFORE(10), 0, 60*10)

2022-12-17T05:37:43.052Z DEBUG [rbot::db::sqlite] update_cache_df 1670391420000000 -> 0
2022-12-17T05:37:43.052Z DEBUG [rbot::db::sqlite] update cache duration 43200000000
2022-12-17T05:37:43.054Z DEBUG [rbot::db::sqlite] create iter 3 microsec
2022-12-17T05:37:52.669Z DEBUG [rbot::db::sqlite] load data before cache df1=(18830237, 4) df2=(2826094, 4)
2022-12-17T05:37:52.671Z DEBUG [rbot::db::df] Select from 1970-01-01T00:00:00.000000 -> 2022-12-14T20:30:00.003000
2022-12-17T05:37:52.686Z DEBUG [rbot::db::df] merge len 18830237
2022-12-17T05:37:52.686Z DEBUG [rbot::db::df] merge df1=(18830237, 4)  df2=(2826094, 4)
2022-12-17T05:37:52.686Z DEBUG [rbot::db::sqlite] cache update diff before 2022-12-07T05:37:00.000000 -> 2022-12-14T20:30:00.000000
2022-12-17T05:37:52.686Z DEBUG [rbot::db::df] ohlcv_df, from=2022-12-07T05:37:00.000000 / to=2022-12-14T20:30:00.000000
2022-12-17T05:37:52.686Z DEBUG [rbot::db::df] Select from 2022-12-07T05:37:00.000000 -> 2022-12-14T20:30:00.000000
2022-12-17T0

In [22]:
ohlcv

,open,high,low,close,volume
timestamp,,,,,
2022-12-07 05:30:00+00:00,17004.96,17010.23,17002.96,17005.82,180.27610
2022-12-07 05:40:00+00:00,17005.82,17017.66,17004.28,17007.46,572.95212
2022-12-07 05:50:00+00:00,17007.39,17013.92,17000.37,17005.57,537.95264
2022-12-07 06:00:00+00:00,17005.17,17008.34,16984.00,16985.61,781.85586
2022-12-07 06:10:00+00:00,16985.60,16993.81,16978.52,16993.36,839.11561
...,...,...,...,...,...
2022-12-15 23:10:00+00:00,17365.84,17368.00,17319.00,17334.55,641.55616
2022-12-15 23:20:00+00:00,17333.81,17351.90,17314.29,17344.47,430.77408
2022-12-15 23:30:00+00:00,17345.37,17345.38,17290.35,17304.33,736.60236


#### OHLCVチャートの描画

`Chart`クラスを利用してチャートを描く。
引数は、`Chart(横幅、縦幅、olhcvデータ(DataFrame型))`となっています。

`Chart`インスタンスを作成し、`show()`メソッドを呼び出すことで描画されます。

描画にはbokehライブラリ`https://bokeh.org`を使っています。また`Chart`クラスはPythonの`site-package`にインストールされているはずですので、それをみながらサブクラスを作ると自由に描画がカスタマイズできると思います。（インストール先の場所の例:`lib/python3.10/site-packages/rbot/chart/`)

In [23]:
chart = Chart(900, 400, ohlcv)
chart.show()

Loading BokehJS ...

#### OHLCVにBackTestの結果を追加する。

`Chart.draw_result`でバックテストの結果を追加表示できます。

In [24]:
chart = Chart(900, 400, ohlcv)

chart.draw_result(back_runner.result)

chart.show()

Loading BokehJS ...

### indicatorの表示

Agentの`on_clock`内で`Agent.log_indicator`を使って保存していたindicatorを`Agent.indicator()`を使って取り出すことができます。`DataFrame`形式になっているので表示します。

In [25]:
diff_low = agent.indicator('diff_low')
diff_high = agent.indicator('diff_high')
range_width = agent.indicator('range_width')
range_width

,value
timestamp,
2022-12-07 05:40:00+00:00,127.286667
2022-12-07 05:50:00+00:00,124.768000
2022-12-07 06:00:00+00:00,145.214400
2022-12-07 06:10:00+00:00,139.889333
2022-12-07 06:20:00+00:00,139.889333
...,...
2022-12-15 23:20:00+00:00,239.940000
2022-12-15 23:30:00+00:00,231.358667
2022-12-15 23:40:00+00:00,229.458667


`Chart.new_figure(新しいパネル名, height=パネルの高さ, title=タイトル)`で新しい描画画面を作ります。

作成したパネル名を指定して、`Chart.line`でデータを表示します。

`Chart.line(描画パネル名, DataFrame, x_key=データフレーム中のx軸名, y_key=データフレーム中のy軸名, color=線の色, legend_label=凡例として表示する名前)`

In [26]:
chart = Chart(900, 400, ohlcv)

chart.draw_result(back_runner.result)

# indicatorの表示
## 新しい描画パネルを作成
chart.new_figure('indicator_panel', height=150, title='indicator')

## indicatorパネルにDataFrameを指定して折線グラフを表示
chart.line('indicator_panel', diff_low, x_key='timestamp', y_key='value', color='#ff0000', legend_label='diff_low')
chart.line('indicator_panel', diff_high, x_key='timestamp', y_key='value', color='#00ff00', legend_label='diff_high')
chart.line('indicator_panel', range_width, x_key='timestamp', y_key='value', color='#0030ff', legend_label='range_width')

chart.show()

Loading BokehJS ...

## 7. まとめ

自分で書くのは百行ぐらいで簡単にバックテストができることが確認できたと思う。

Biannceしか対応していないとほほな状況ですが使ってみてご意見お聞かせください。
Twitter(@yasstake)でもgithubでissueあげてもOK.

人気になったらDiscord立ち上げてみたいなー。



## 8. Next Step

いろいろバックテストしてみよう

#### BreakOutAgentのパラメータ変更
以下のパラメータを変えたらどうなるか確認してみよう
1) パラメータK (現在は1.6きめうち)
2) 指値（現在はbestプライス。
   1) buyのときに低い価格で指すと利益は上がるがExpire率が上がる。
   2) 高い価格で指すとExpire率が下がるが利益が下がる。（この場合、本来はTakerFeeが取られるが現在はMakerFeeで計算）
3) ロット（現在は0.01 BTC)の変更
4) バックテスト期間の延長（現在は１０日）　（期間を変更するとバックテスト中にエラーがでることがあります。その場合はPythonカーネルの再起動を行なってください）

さらに独自のIndicatorをつくり、独自ロジックをつくってバックテストしてみよう（OHLCVをベースに判断するロジックならば問題なく作れるはず）